## 使用 Gluon 实现RNN, 创作歌词
### 1. 为什么简介实现比从零实现快这么多
- "简介实现"每次迭代 都会从计算图分离隐含状态, 导致模型梯度参数只依赖当前的批量序列, 从而减小 每次迭代的计算开销

### 2. 





In [1]:
! pip install mxnet-cu101mkl
from google.colab import drive
gd_path = '/content/drive'
drive.mount(gd_path)
%cd '/content/drive/My Drive/Colab Notebooks/dive_into_deep_learning'
%pwd

     |████████████████████████████████| 587.7MB 28kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/dive_into_deep_learning


'/content/drive/My Drive/Colab Notebooks/dive_into_deep_learning'

In [0]:
import re
import math
import time
import zipfile
import traceback
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn

In [0]:
# 加载数据集
with zipfile.ZipFile('DataResources/Chapter_6/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars = re.sub(r'\s+', ' ', corpus_chars)
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in corpus_chars]

In [0]:
# 定义 RNN 模型
def gen_net(num_hiddens, batch_size):
    _rnn = rnn.RNN(num_hiddens)
    _rnn.initialize()
    _rnn.begin_state(batch_size=batch_size)
    return _rnn
    
class RNN(nn.Block):
    def __init__(self, num_hiddens, batch_size, vocab_size, **kwargs):
        super(RNN, self).__init__(**kwargs)
        self.rnn = gen_net(num_hiddens, batch_size)
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)  # 为什么需要一个全连接层
    
    def forward(self, inputs, state):
        # 将输入转置成(num_steps, batch_size)后获取one-hot向量表示
        X = nd.one_hot(inputs.T, self.vocab_size)
        Y, state = self.rnn(X, state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.reshape((-1, Y.shape[-1])))
        return output, state
    
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

In [0]:
temp_net = gen_net(100, 2)

In [6]:
init_state = temp_net.begin_state(batch_size=2)
X = nd.random.uniform(shape=(30, 2, vocab_size))
Y, state_new = temp_net(X, init_state)
(Y.shape, state_new)

((30, 2, 100), [
  [[[0.         0.         1.0604693  0.7939049  0.         0.
     0.40115237 0.         0.         0.         1.653013   0.
     0.         2.4500334  0.         1.7684158  0.5570866  0.
     0.         0.         0.         1.8967252  1.520531   0.16472216
     0.31395215 0.         0.         0.44583786 0.         0.
     2.8692446  0.         0.         0.         1.166519   0.
     0.         0.         0.         0.         0.         0.7190442
     0.         3.140932   0.         0.61330855 0.         0.63296187
     0.5235083  0.56653047 1.4101675  0.8184509  0.19092263 0.
     0.98904157 0.         0.         0.2568788  0.09498417 0.
     0.         1.6761665  0.         1.2460227  0.         1.1604192
     0.         1.5758954  0.         0.         0.41418585 1.0927155
     0.84002507 1.0490992  1.5553081  0.         0.75275314 0.
     2.5894449  0.597352   0.37891552 1.0617347  1.4261123  0.8206196
     0.10223021 0.65250546 0.         0.         0.630450

In [0]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char, char_to_idx):
    """
    预测prefix 之后的歌词
    """
    state = model.begin_state(batch_size=1, ctx=ctx)  # 使用model的成员函数来初始化隐藏状态
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1, 1))
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [0]:
def try_gpu(gpu_number=0):
    """
        Return gpu(i) if exists, otherwise return cpu().
    """
    try:
        _ = mx.nd.array([1, 2, 3], ctx=mx.gpu(gpu_number))
        print("Try GPU: {}".format(gpu_number))
    except mx.MXNetError:
        print("Try CPU: {}".format(gpu_number))
        return mx.cpu()
    return mx.gpu(gpu_number)

In [9]:
ctx = try_gpu()
model = RNN(256, 2, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('分开', 10, model, vocab_size, ctx, idx_to_char, char_to_idx)

Try GPU: 0


'分开寇褆孔严统昏铩印抢声'

In [0]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    """
    相邻采样: 相邻 epoch 的 batch_size 样本是相邻的
    Sample mini-batches in a consecutive order from sequential data.
    """
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0 : batch_size * batch_len].reshape((
        batch_size, batch_len))  # 只要 前面的batch_size * batch_len 个 
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i : i + num_steps]
        Y = indices[:, i + 1 : i + num_steps + 1]
        yield X, Y

In [0]:
# 剪裁梯度
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [0]:
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    perplexity_hist = list()
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd',
                            {'learning_rate': lr, 'momentum': 0, 'wd': 0})

    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(
            corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        for X, Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # 梯度裁剪
            params = [p.data() for p in model.collect_params().values()]
            grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)  # 因为已经误差取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size
        perplexity = math.exp(l_sum / n)
        perplexity_hist.append(perplexity)
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(prefix, pred_len, model, vocab_size, ctx, idx_to_char, char_to_idx))
    return perplexity_hist

In [13]:
num_hiddens, num_steps = 256, 35
num_epochs, batch_size, lr, clipping_theta = 300, 32, 100, 0.01
pred_period, pred_len, prefixes = 10, 50, ['分开', '不分开', '我静静地']
perplexities = train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 10, perplexity 296.871192, time 0.51 sec
 - 分开 我们 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我
 - 不分开 我们的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 我 我的 
 - 我静静地 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我的爱 我
epoch 20, perplexity 170.466381, time 0.53 sec
 - 分开 不用麻烦 我们不是 你的灵界 在我的爱泪 在我的手气 我们了着 一个一直 我们了感 你的世界 在我
 - 不分开 我们不能 你的世界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界
 - 我静静地 你的眼界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界 你的爱界
epoch 30, perplexity 97.244393, time 0.50 sec
 - 分开 我们 如些我 娘中 人 Ya 一切 me 我用 我们 我不光 不是我 爱着不人 你的灵魂 你我等的
 - 不分开 不是我 的手魂人 一个界 的灵魂只剩下一种 等空的客倾 我们上的世界 在空地 那子 我在 我很 我
 - 我静静地 你的眼溃 你我等不到 你的眼界 你的等里 你我的感情 我在等待 你就会 你手 我们 我不光 不是我
epoch 40, perplexity 59.051912, time 0.50 sec
 - 分开 我们 如单我 想手 那里 豆腐 豆e 赶e 功紧 功紧 功e 功紧 功紧 功紧 功紧 功紧 功紧 
 - 不分开 不是我们在泪 会有我们的泪诺 能说你说 我有你 对手 你说 我有你 爱情 我 再感了 一直一步 等
 - 我静静地 你说不该 让我们乘着阳光 看着远方河个天 说我的善 在你比外的溪界 你说你方越 天涯的手笑 我在等
epoch 50, perplexity 39.258070, time 0.50 sec
 - 分开 我们在感力 我在赶一间 你的年如 我不想 你了那果 你始了 不来成 你在空 对远了 你在空 对远的
 - 不分开 不是我们的天 下一点 原a 不用麻烦了 不

In [14]:
perplexities

[2167.2273518627203,
 514.4585204487433,
 457.8852835246202,
 420.7862561515121,
 383.6545682777267,
 364.0001782071033,
 343.5226008965296,
 328.82278618159575,
 311.83496446161996,
 297.02807612591255,
 280.49930126415984,
 267.01795807534313,
 251.88793508749058,
 239.4567273105404,
 225.32562834593836,
 214.1074965143991,
 201.56094712420864,
 191.8478823451365,
 180.2680511928871,
 170.34128430721054,
 161.63030297139656,
 152.27629063780228,
 143.78831618339515,
 136.47848126325633,
 128.67948552373173,
 121.8635101857824,
 114.67635529679038,
 108.4786249810788,
 102.94673360006074,
 97.56443789352444,
 92.00525670630391,
 87.49055426495383,
 82.07857585383975,
 78.59403105167628,
 74.01850914776175,
 71.5366791878533,
 67.17857936204908,
 63.81090131675539,
 61.52943158808321,
 58.274185760430285,
 55.96988497320034,
 53.58049011531275,
 51.0634553433627,
 49.10944349001716,
 46.80599271383927,
 44.91437242817392,
 42.89524223209852,
 41.97433381108114,
 39.82842817075176,
 38.